In [ ]:
"""
1. Load faces from museum data set
2. Load base_model that will be transfered
    2.1. Freeze its layers
3. Build new model on top of it
4. Train new model with museum data set
5. TODO: Fine tune?
"""

In [2]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from fairness import fairnessMetrics as fm

In [3]:
# 
def readImages(folderName, image_size, batch_size):
    train_ds = keras.preprocessing.image_dataset_from_directory(
        folderName,
        validation_split=0.2,
        subset="training",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size
    )
    val_ds = keras.preprocessing.image_dataset_from_directory(
        folderName,
        validation_split=0.2,
        subset="validation",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size
    )
    return (train_ds, val_ds)

In [8]:
metric_dict = {
                'TP': tf.keras.metrics.TruePositives(), 
                'TN': tf.keras.metrics.TrueNegatives(),
                'FP': tf.keras.metrics.FalsePositives(), 
                'FN': tf.keras.metrics.FalseNegatives(), 
                'TPR': fm.TruePositiveRate(),
                'TNR': fm.TrueNegativeRate(),
                'FPR': fm.FalsePositiveRate(),
                'FNR': fm.FalseNegativeRate(),
                'PPV': fm.PositivePredictedValue(),
                'FDR': fm.FalseDiscoveryRate(),
                'NPV': fm.NegativePredictedValue(),
                'FOR': fm.FalseOmissionRate(),
                'BDPD': fm.BinaryDemographicParityDiff(),
                'DP': fm.DemographicParity(),
                'BEOD': fm.BinaryEqualizedOddsDiff(),
                'BPPD': fm.BinaryProportionalParityDiff(),
                'PP': fm.ProportionalParity(),
                'BPRPD': fm.BinaryPredictiveRateParityDiff(),
                'PRP': fm.PredictiveRateParity(),
                'BAPD': fm.BinaryAccuracyParityDiff(),
                'AP': fm.AccuracyParity(),
                'BFNRPD': fm.BinaryFalseNegativeRateParityDiff(),
                'BFPRPD': fm.BinaryFalsePositiveRateParityDiff(),
                'BNPRPD': fm.BinaryNegativePredictiveRateParityDiff(),
                'NPRP': fm.NegativePredictiveRateParity(),
                'BSPD': fm.BinarySpecificityParityDiff()
}

dict

In [24]:
# Load pretrained base model.
# Loading with the custom metrics leads to problems if this model is going to be trained further. 
# Will it be a problem when fine tuning?
base_model = tf.keras.models.load_model('saved_base_model', compile = False, custom_objects = metric_dict) 

# Freeze the base_model
base_model.trainable = False

In [25]:
"""
Create new model on top of base model
Replace top layer 
"""
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [26]:
new_model = keras.Model(base_model.input, base_model.layers[-5].output)
new_model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [27]:
inputs = keras.Input(shape=(224, 224, 3))

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here.
x = new_model(inputs, training=False)

# What layers do we want to add?
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout

outputs = keras.layers.Dense(1)(x)
new_model = keras.Model(inputs, outputs)

new_model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
model_5 (Functional)         (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


In [28]:
image_size = (224, 224) # This step should be standardizing, should be able to be changed to what is approptiate
batch_size = 2
data = readImages("museum", image_size, batch_size)

Found 4897 files belonging to 2 classes.
Using 3918 files for training.
Found 4897 files belonging to 2 classes.
Using 979 files for validation.


In [29]:
metrics_list = ["accuracy",
                tf.keras.metrics.TruePositives(), 
                tf.keras.metrics.TrueNegatives(),
                tf.keras.metrics.FalsePositives(), 
                tf.keras.metrics.FalseNegatives(), 
                fm.TruePositiveRate(),
                fm.TrueNegativeRate(),
                fm.FalsePositiveRate(),
                fm.FalseNegativeRate(),
                fm.PositivePredictedValue(),
                fm.FalseDiscoveryRate(),
                fm.NegativePredictedValue(),
                fm.FalseOmissionRate(),
                fm.BinaryDemographicParityDiff(),
                fm.DemographicParity(),
                fm.BinaryEqualizedOddsDiff(),
                fm.BinaryProportionalParityDiff(),
                fm.ProportionalParity(),
                fm.BinaryPredictiveRateParityDiff(),
                fm.PredictiveRateParity(),
                fm.BinaryAccuracyParityDiff(),
                fm.AccuracyParity(),
                fm.BinaryFalseNegativeRateParityDiff(),
                fm.BinaryFalsePositiveRateParityDiff(),
                fm.BinaryNegativePredictiveRateParityDiff(),
                fm.NegativePredictiveRateParity(),
                fm.BinarySpecificityParityDiff()]

In [31]:
metrics_list = ["accuracy"]

In [ ]:
# Train new model
new_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=metrics_list,
)

epochs = 3
new_model.fit(data[0], epochs=epochs, validation_data=data[1])

Epoch 1/3
  88/1959 [>.............................] - ETA: 14:25 - loss: 0.6930 - accuracy: 0.5455